In [ ]:
from api_request import api_request
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
from io import StringIO

In [ ]:
# read indicator list
cl_ind = pd.read_csv("./indicator_list/CL_INDICATORS_TRANSMONEE.csv")
# read ecaro country list
ecaro_geo = pd.read_csv("./indicator_list/CL_COUNTRY.csv")

In [ ]:
# dropdown indicators
dd_ind = dcc.Dropdown(
    id="my_ind",
    options=[
        {"label": value, "value": key}
        for key, value in zip(cl_ind.Id,cl_ind.Name)
    ],
    value='NT_ANT_WHZ_PO2'
)
# dropdown countries
dd_geo = dcc.Dropdown(
    id="my_geo",
    options=[
        {"label": value, "value": key}
        for key, value in zip(ecaro_geo.Id,ecaro_geo.Name)
    ],
    value='ALB'
)

In [ ]:
# detect proxy configuration for JupyterHub or Binder
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
# App Layout
app.layout = html.Div([
    html.H2("Check TM DB latests observations"),
    html.H6("Browse by Indicator and Country to get the latest year reported"),
    html.Div([
        html.Div(
            ["Select Indicator", dd_ind],
            style={'width': '49%', 'display': 'inline-block'},
        ),
        html.Div(
            ["Select Country (None for All)", dd_geo],
            style={'width': '49%', 'display': 'inline-block'},
        ),
    ]),
    html.Br(),
    html.Div(id="my-output")
])

In [ ]:
# parameters: API-SDMX Helix (latest observation)
api_params = {"lastNObservations":1}
# API headers
api_headers = {
    "Accept": "application/vnd.sdmx.data+csv;version=1.0.0",
    "Accept-Encoding": "gzip",
}
# UNICEF data warehouse TM url endpoint
wh_url = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE"

In [ ]:
# Define callback to update graph (output/input: 'id', 'property')
@app.callback(
    Output("my-output", "children"),
    Input("my_ind", "value"),
    Input("my_geo", "value"),
)
def query_db(indicator, country):
    if country:
        query = f"{country}.{indicator}...."
    else:
        query = f".{indicator}...."
    ind_geo_req = api_request(f"{wh_url}/{query}", api_params, api_headers)
    if ind_geo_req.status_code == 200:
        data = pd.read_csv(StringIO(ind_geo_req.text))
        maxdata = "Latest Year: {}".format(data["TIME_PERIOD:Time period"].max())
    else:
        maxdata = "No Data"
    return maxdata

In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')